In [1]:
# prado_users = pd.read_csv("data/Prado_users_emotions_OnePolarity.csv")

# prado_users = prado_users.replace('USA','United States')
# prado_users = prado_users.replace('Argenitina','Argentina')
# prado_users = prado_users.replace('<12', '0-12')
# prado_users = prado_users.replace('>70', '70-100')

# prado_users.to_csv("data/Prado_users_emotions_OnePolarity.csv")

In [2]:
import pandas as pd

from src.SimilarityUsers import SimilarityUsers

from src.UsersClustering import UsersClustering
from src.AverageUser import AverageUser

In [3]:
users = pd.read_csv("data/Prado_users_emotions_OnePolarity.csv")
sim_cuadros = pd.read_csv("data/Prado_artworks_similarity.csv", index_col=0)

In [14]:
df = SimilarityUsers(users, sim_cuadros).getSimilarityMatrix()
df

,1,2,3,4,5,6,7,8,9,10,...,270,275,277,288,289,298,299,300,301,306
1,1.000000,0.35965,0.543583,0.538550,0.762800,0.543900,0.527158,0.413467,0.771017,0.44780,...,0.08410,0.601450,0.45965,0.532700,0.475300,0.410350,0.663950,0.42105,0.58355,0.55340
2,0.359650,1.00000,0.513300,0.501450,0.399950,0.614950,0.458450,0.364550,0.368700,0.33435,...,0.45625,0.500000,0.90000,0.568750,0.468750,0.387500,0.437500,0.83120,0.66870,0.80625
3,0.543583,0.51330,1.000000,0.766617,0.591376,0.667969,0.529665,0.491186,0.575324,0.42451,...,0.31030,0.459664,0.41330,0.528414,0.408271,0.203283,0.397164,0.60200,0.43950,0.31955
4,0.538550,0.50145,0.766617,1.000000,0.613588,0.619467,0.554450,0.532375,0.594100,0.44685,...,0.31600,0.492000,0.40145,0.560750,0.377250,0.171250,0.429500,0.63725,0.47475,0.30770
5,0.762800,0.39995,0.591376,0.613588,1.000000,0.566371,0.460092,0.432733,0.848617,0.55460,...,0.14150,0.586600,0.49995,0.517850,0.534800,0.418750,0.586600,0.56875,0.66875,0.46870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.410350,0.38750,0.203283,0.171250,0.418750,0.218750,0.197100,0.241050,0.450000,0.34035,...,0.51025,0.487500,0.48750,0.418750,0.518750,1.000000,0.550000,0.38750,0.55000,0.51875
299,0.663950,0.43750,0.397164,0.429500,0.586600,0.551750,0.407633,0.337150,0.594267,0.36855,...,0.29375,0.937500,0.53750,0.868750,0.801750,0.550000,1.000000,0.43750,0.60000,0.56875
300,0.421050,0.83120,0.602000,0.637250,0.568750,0.666150,0.451550,0.390817,0.537500,0.33435,...,0.45625,0.500000,0.73120,0.568750,0.468750,0.387500,0.437500,1.00000,0.83750,0.63745
301,0.583550,0.66870,0.439500,0.474750,0.668750,0.566150,0.414050,0.290817,0.700000,0.43435,...,0.29375,0.537500,0.76870,0.468750,0.568750,0.550000,0.600000,0.83750,1.00000,0.79995


In [32]:
%%capture output
sim_users = pd.read_csv("data/Prado_users_similarity_OnePolarity.csv", index_col=0)
users = pd.read_csv('data/Prado_users_emotions_OnePolarity.csv', index_col=0)

users['positive'] = users['positive'].apply(eval)
users['negative'] = users['negative'].apply(eval)
users['mixed'] = users['mixed'].apply(eval)

labels = UsersClustering().kMedoidsFromMatrix(sim_users)
users['cluster'] = labels

df = AverageUser().computeAverageUser(users)

In [33]:
df

,userId,age,gender,country,positive,negative,mixed,cluster
0,0,45-54,male,Spain,"[57727c95edc2cb3880e4d1d4, 57726f59edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57726f49edc2cb3880b...","[57727015edc2cb3880bceb08, 5772752eedc2cb3880c...",1
1,0,25-34,female,Spain,[],"[57727cbaedc2cb3880e561f7, 57726f59edc2cb3880b...",[57727015edc2cb3880bceb08],5
2,0,45-54,female,Spain,"[57726f59edc2cb3880ba76b3, 57727c95edc2cb3880e...","[57727cbaedc2cb3880e561f7, 57726e45edc2cb3880b...","[57727cbaedc2cb3880e561f7, 577278d5edc2cb3880d...",4
3,0,35-44,male,United States,"[57726f59edc2cb3880ba76b3, 57727c95edc2cb3880e...","[57726e37edc2cb3880b628ee, 5772700eedc2cb3880b...","[57727cbaedc2cb3880e561f7, 57727015edc2cb3880b...",0
4,0,25-34,male,Spain,"[57726f59edc2cb3880ba76b3, 57727c95edc2cb3880e...","[57726f4fedc2cb3880ba0b6d, 57727015edc2cb3880b...","[57726f59edc2cb3880ba76b3, 57727015edc2cb3880b...",2
5,0,35-44,male,Spain,[],"[57727cbaedc2cb3880e561f7, 57726f59edc2cb3880b...",[57727015edc2cb3880bceb28],3
6,0,35-44,female,Spain,"[57726f59edc2cb3880ba76b3, 57727c95edc2cb3880e...",[],[577278cdedc2cb3880d9370c],6


In [2]:
import pandas as pd
import src.AverageUser as AU

In [3]:
users = pd.read_csv('data/Prado_users_clusters_OnePolarity.csv')
emotions = pd.read_csv('data/Prado_users_emotions_OnePolarity.csv')
users['positive'] = emotions['positive'].apply(eval)
users['negative'] = emotions['negative'].apply(eval)
users['mixed'] = emotions['mixed'].apply(eval)
AU.AverageUser().computeAverageUser(users)

,userId,age,gender,country,medoid,positive,negative,mixed,cluster
0,0,18-24,male,Spain,-1.0,"[57726f59edc2cb3880ba76b3, 57727c95edc2cb3880e...","[57727cbaedc2cb3880e561f7, 577278d5edc2cb3880d...","[57727015edc2cb3880bceb08, 5772700eedc2cb3880b...",0
1,0,35-44,female,Spain,-1.0,"[57726f59edc2cb3880ba76b3, 57727015edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57726f59edc2cb3880b...","[57727071edc2cb3880be3213, 57727015edc2cb3880b...",1
2,0,18-24,female,Other,-1.0,"[57726f59edc2cb3880ba76b3, 5772752eedc2cb3880c...","[57727cbaedc2cb3880e561f7, 57726f76edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57727015edc2cb3880b...",2
3,0,45-54,male,Other,-1.0,"[57727c95edc2cb3880e4d1d4, 57726f59edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57726f49edc2cb3880b...","[57727015edc2cb3880bceb08, 57726f59edc2cb3880b...",3
4,0,35-44,female,Spain,-1.0,"[57726f59edc2cb3880ba76b3, 57727015edc2cb3880b...","[57726f4fedc2cb3880ba0b6d, 5772752eedc2cb3880c...","[57727cbaedc2cb3880e561f7, 57726f76edc2cb3880b...",4


In [1]:
%%capture output
!pip install ipywidgets
!pip install jupyterlab_widgets
# !pip install nodejs
# !pip install npm
!pip install widgetsnbextension
!jupyter nbextension enable --py widgetsnbextension

In [2]:
import ipywidgets as widgets

In [3]:
w = widgets.IntSlider()

In [4]:
display(w)

IntSlider(value=0)

In [5]:
display(w)

IntSlider(value=0)

In [6]:
a = widgets.FloatText()
b = widgets.FloatSlider()
display(a,b)

mylink = widgets.jslink((a, 'value'), (b, 'value'))

FloatText(value=0.0)

FloatSlider(value=0.0)

In [7]:
widgets.Dropdown(
    options=['1', '2', '3'],
    value='1',
    description='Number:',
    disabled=False,
)

Dropdown(description='Number:', options=('1', '2', '3'), value='1')

In [8]:
from os import listdir
from os.path import isfile, join

mypath = 'data'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

filesDropdown = widgets.Dropdown(
    options=files,
    value=files[0],
    description='Users:',
    disabled=False,
)
itemsDropdown = widgets.Dropdown(
    options=files,
    value=files[0],
    description='Items:',
    disabled=False,
)

display(filesDropdown)
display(itemsDropdown)

Dropdown(description='Users:', options=('artworkColors.json', 'artworkEmotions.json', 'Clean_users.csv', 'coun…

Dropdown(description='Items:', options=('artworkColors.json', 'artworkEmotions.json', 'Clean_users.csv', 'coun…

In [10]:
filesDropdown.value

'Prado_users_similarity.csv'